<a href="https://colab.research.google.com/github/firarru/ML-Manual-Naive-Bayes/blob/main/CAD_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nabila Zalfa Shafira

1206220006/DS 02-01

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
df = pd.read_excel('https://github.com/firarru/ML-Manual-Naive-Bayes/raw/refs/heads/main/Dataset/CAD.xlsx')
df

,No,Age,Weight,Length,Sex,BMI,DM,HTN,Current Smoker,EX-Smoker,...,K,Na,WBC,Lymph,Neut,PLT,EF-TTE,Region RWMA,VHD,Cath
0,1,53,90,175,Male,29.387755,0,1,1,0,...,4.7,141,5700,39,52,261,50,0,N,Cad
1,2,67,70,157,Fmale,28.398718,0,1,0,0,...,4.7,156,7700,38,55,165,40,4,N,Cad
2,3,54,54,164,Male,20.077335,0,0,1,0,...,4.7,139,7400,38,60,230,40,2,mild,Cad
3,4,66,67,158,Fmale,26.838648,0,1,0,0,...,4.4,142,13000,18,72,742,55,0,Severe,Normal
4,5,50,87,153,Fmale,37.165193,0,1,0,0,...,4.0,140,9200,55,39,274,50,0,Severe,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,299,58,84,168,Male,29.761905,0,0,0,0,...,4.8,146,8500,34,58,251,45,0,N,Cad
299,300,55,64,152,Fmale,27.700831,0,0,0,0,...,4.0,139,11400,16,80,377,40,0,mild,Normal
300,301,48,77,160,Fmale,30.078125,0,1,0,0,...,4.0,140,9000,35,55,279,55,0,N,Normal
301,302,57,90,159,Fmale,35.599858,1,0,0,0,...,3.8,141,3800,48,40,208,55,0,N,Normal


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 56 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   No                     303 non-null    int64  
 1   Age                    303 non-null    int64  
 2   Weight                 303 non-null    int64  
 3   Length                 303 non-null    int64  
 4   Sex                    303 non-null    object 
 5   BMI                    303 non-null    float64
 6   DM                     303 non-null    int64  
 7   HTN                    303 non-null    int64  
 8   Current Smoker         303 non-null    int64  
 9   EX-Smoker              303 non-null    int64  
 10  FH                     303 non-null    int64  
 11  Obesity                303 non-null    object 
 12  CRF                    303 non-null    object 
 13  CVA                    303 non-null    object 
 14  Airway disease         303 non-null    object 
 15  Thyroi

Kita pisahkan variabel dependen(target) dan variabel independen. Dalam kasus ini, variabel dependen merupakan fitur Cath dan fitur yang lain merupakan variabel independen

In [4]:
X = df.loc[:, df.columns != 'Cath'] #fitur
Y = df['Cath']  # target

In [5]:
df['Cath'].unique()

array(['Cad', 'Normal'], dtype=object)

In [6]:
print("CAD :", df[df['Cath'] == 'Cad'].shape[0])
print("Normal :", df[df['Cath'] == 'Normal'].shape[0])

CAD : 216
Normal : 87


In [7]:
label_encoders = {}
for column in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])
    label_encoders[column] = le

<ipython-input-7-8e0b8b1147e2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = le.fit_transform(X[column])
<ipython-input-7-8e0b8b1147e2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = le.fit_transform(X[column])
<ipython-input-7-8e0b8b1147e2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [8]:
y = LabelEncoder().fit_transform(Y)  # CAD = 1, Normal = 0

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

# **Tanpa Scikit-Learn**

Hitung prior probability untuk setiap kelas variable target

In [10]:
def calculate_prior(y_train):
    unique_classes, counts = np.unique(y_train, return_counts=True)
    priors = {cls: count / len(y_train) for cls, count in zip(unique_classes, counts)}
    return priors

Hitung mean dan variance untuk setiap fitur per kelas

In [11]:
def calculate_mean_variance(x_train, y_train):
    summary = {}
    classes = np.unique(y_train)

    for cls in classes:
        class_data = x_train[y_train == cls]  # Data dengan kelas tertentu
        class_summary = {
            col: (class_data[col].mean(), class_data[col].var() + 1e-3)  # Tambah smoothing kecil
            for col in x_train.columns
        }
        summary[cls] = class_summary

    return summary

Hitung likelihood dengan distribusi Gaussian

In [12]:
def gaussian_probability(x, mean, var):
    exponent = exp(-((x - mean) ** 2) / (2 * var))
    return (1 / sqrt(2 * pi * var)) * exponent

Hitung posterior probability (P(Class|Data)) dan prediksi kelas

In [13]:
def predict_naive_bayes(x_row, summary, priors):
    probabilities = {}

    for cls, class_summary in summary.items():
        probabilities[cls] = log(priors[cls])  # log P(Class)

        for feature, (mean, var) in class_summary.items():
            # probabilities[cls] *= gaussian_probability(x_row[feature], mean, var)  # Likelihood P(X|Class)
            prob = gaussian_probability(x_row[feature], mean, var)
            probabilities[cls] += log(prob) if prob > 0 else -100  # Menghindari log(0)

    return max(probabilities, key=probabilities.get)  # Pilih kelas dengan probabilitas tertinggi

Latih model dengan data training

In [14]:
priors = calculate_prior(y_train)
summary = calculate_mean_variance(x_train, y_train)

Prediksi untuk baris terakhir

In [15]:
from math import sqrt, pi, exp, log
last_row = X.iloc[[-1]].to_dict(orient="records")[0]  # Ambil row terakhir sebagai dictionary
predicted_class = predict_naive_bayes(last_row, summary, priors)

Konversi hasil prediksi ke label asli

In [16]:
actual_category = df["Cath"].iloc[-1]  # Label asli dari dataset
predicted_label = "CAD" if predicted_class == 1 else "Normal"

In [17]:
print(f"Kategori Asli: {actual_category}")
print(f"Prediksi Model: {predicted_label}")

Kategori Asli: Cad
Prediksi Model: CAD


In [18]:
# Prediksi untuk semua data test
def predict_all(X_test, summary, priors):
    return np.array([predict_naive_bayes(row, summary, priors) for _, row in x_test.iterrows()])

y_pred_manual = predict_all(x_test, summary, priors)

Metriks Evaluasi

In [19]:
accuracy_manual = accuracy_score(y_test, y_pred_manual)
precision_manual = precision_score(y_test, y_pred_manual, average='weighted')
recall_manual = recall_score(y_test, y_pred_manual, average='weighted')
f1_manual = f1_score(y_test, y_pred_manual, average='weighted')

print("\n=== Evaluasi Model Naïve Bayes Tanpa Scikit-Learn ===")
print(f"Akurasi      : {accuracy_manual:.2%}")
print(f"Precision    : {precision_manual:.2%}")
print(f"Recall       : {recall_manual:.2%}")
print(f"F1-Score     : {f1_manual:.2%}")


=== Evaluasi Model Naïve Bayes Tanpa Scikit-Learn ===
Akurasi      : 62.30%
Precision    : 80.73%
Recall       : 62.30%
F1-Score     : 63.58%


Naïve Bayes tanpa Scikit-Learn sudah cukup baik dengan 75.41% akurasi, tetapi model masih bisa ditingkatkan dalam mendeteksi pasien CAD dengan lebih akurat. Precision tinggi menunjukkan model tidak terlalu sering salah mendeteksi CAD pada pasien normal, tetapi recall bisa ditingkatkan agar lebih sedikit kasus CAD yang terlewat

# **Dengan Scikit-Learn**

In [20]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [21]:
model = GaussianNB(var_smoothing=1e-9)

In [22]:
model.fit(x_train, y_train)

GaussianNB()

In [23]:
y_pred_sklearn = model.predict(x_test)

In [24]:
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
precision_sklearn = precision_score(y_test, y_pred_sklearn, average='weighted')
recall_sklearn = recall_score(y_test, y_pred_sklearn, average='weighted')
f1_sklearn = f1_score(y_test, y_pred_sklearn, average='weighted')
print("\n=== Evaluasi Model Naïve Bayes Scikit-Learn ===")
print(f"Akurasi      : {accuracy_sklearn:.2%}")
print(f"Precision    : {precision_sklearn:.2%}")
print(f"Recall       : {recall_sklearn:.2%}")
print(f"F1-Score     : {f1_sklearn:.2%}")


=== Evaluasi Model Naïve Bayes Scikit-Learn ===
Akurasi      : 72.13%
Precision    : 81.41%
Recall       : 72.13%
F1-Score     : 73.57%


Model cukup baik dengan akurasi 72.13%, tetapi masih bisa ditingkatkan terutama dalam mendeteksi pasien CAD yang benar-benar positif (Recall masih bisa diperbaiki). Precision tinggi (81.41%) menunjukkan model tidak sering salah mendeteksi CAD pada pasien yang sebenarnya normal. F1-Score 73.57% menunjukkan keseimbangan, tetapi recall masih perlu ditingkatkan untuk menghindari pasien CAD yang tidak terdeteksi.

# **Kesimpulan**

Model pertama, yaitu Naïve Bayes tanpa scikit-learn memiliki performa lebih tinggi dibandingkan model kedua (Naive Bayes scikit-learn) dalam semua metrik evaluasi (akurasi, precision, recall, dan F1-score). Hal tersebut bisa disebabkan karena adanya Log-probability yang membantu mencegah underflow pada model pertama